In [2]:
import kimports; reload(kimports); from kimports import *
import kutils; reload(kutils)

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


<module 'kutils' from 'kutils.pyc'>

In [3]:
path = '../data/shapes'

In [4]:
bn_model = keras.models.Sequential([
        keras.layers.normalization.BatchNormalization(input_shape=(3, 100,100)),
    ])

In [5]:
model = keras.models.Sequential([
       
        keras.layers.convolutional.Convolution2D(64, 3, 3, activation='relu', input_shape=bn_model.layers[0].output_shape[1:]),
        keras.layers.MaxPooling2D((2,2), strides=(2,2)),
        keras.layers.core.Dropout(0.4),
        keras.layers.core.Flatten(),
        keras.layers.core.Dense(4096, activation='relu'),
        keras.layers.core.Dropout(0.4),
        #keras.layers.core.Dense(4096, activation='relu'),
        keras.layers.core.Dense(2, activation='softmax')
    ])

In [6]:
bn_model.compile(
    optimizer=keras.optimizers.Adam(lr=0.00001),
    loss='categorical_crossentropy',
    metrics=['accuracy'])

In [7]:
model.compile(
    optimizer=keras.optimizers.Adam(lr=0.00001),
    loss='categorical_crossentropy',
    metrics=['accuracy'])

In [8]:
train_batches = keras.preprocessing.image.ImageDataGenerator(
    ).flow_from_directory(path+'/train', target_size=(100,100), batch_size=64)
valid_batches = keras.preprocessing.image.ImageDataGenerator(
    ).flow_from_directory(path+'/valid', target_size=(100,100), batch_size=64)

Found 1013 images belonging to 2 classes.
Found 247 images belonging to 2 classes.


In [9]:
bn_train_features = bn_model.predict_generator(train_batches, train_batches.nb_sample)

In [10]:
bn_valid_features = bn_model.predict_generator(valid_batches, valid_batches.nb_sample)

In [15]:
bn_model.predict_generator?

In [13]:
model.fit(bn_train_features, kutils.onehot(train_batches),
         validation_data=(bn_valid_features, kutils.onehot(valid_batches)))

Train on 1013 samples, validate on 247 samples
Epoch 1/10
1013/1013 [==============================] - 17s - loss: 6.5733 - acc: 0.5913 - val_loss: 6.3298 - val_acc: 0.6073
Epoch 2/10
1013/1013 [==============================] - 17s - loss: 6.4281 - acc: 0.6012 - val_loss: 6.3298 - val_acc: 0.6073
Epoch 3/10
 928/1013 [==========================>...] - ETA: 1s - loss: 6.3743 - acc: 0.6045

KeyboardInterrupt: 

In [6]:
model.fit(
    train_batches,
    train_batches.nb_sample,
    20,
    validation_data=valid_batches,
    nb_val_samples=valid_batches.nb_sample)

Epoch 1/20
1013/1013 [==============================] - 12s - loss: 0.8559 - acc: 0.5696 - val_loss: 1.1015 - val_acc: 0.5709
Epoch 2/20
1013/1013 [==============================] - 11s - loss: 0.5864 - acc: 0.6900 - val_loss: 0.8488 - val_acc: 0.5385
Epoch 3/20
1013/1013 [==============================] - 11s - loss: 0.5291 - acc: 0.7414 - val_loss: 0.7812 - val_acc: 0.6235
Epoch 4/20
1013/1013 [==============================] - 11s - loss: 0.4605 - acc: 0.7759 - val_loss: 0.6867 - val_acc: 0.6316
Epoch 5/20
1013/1013 [==============================] - 11s - loss: 0.4061 - acc: 0.8164 - val_loss: 0.8179 - val_acc: 0.5425
Epoch 6/20
1013/1013 [==============================] - 11s - loss: 0.3513 - acc: 0.8766 - val_loss: 0.7169 - val_acc: 0.6275
Epoch 7/20
1013/1013 [==============================] - 11s - loss: 0.3281 - acc: 0.8746 - val_loss: 0.7688 - val_acc: 0.5709
Epoch 8/20
1013/1013 [==============================] - 11s - loss: 0.3082 - acc: 0.8806 - val_loss: 0.7354 - val_acc:

In [7]:
cl = model.layers[1]

In [8]:
r = np.matrix([[-1,-1,-1], [1,1,1], [0,0,0]])

In [9]:
unrolled = []
for convw in cl.get_weights()[0]:
    for chanw in convw:
        unrolled += [chanw, np.rot90(chanw, 1), np.rot90(chanw, 2), np.rot90(chanw, 3)]

In [10]:
ssq = map(lambda (i, u): (i, np.sum(np.power(r - u/np.mean(np.abs(u)), 2))), enumerate(unrolled))

In [11]:
sorted(ssq, key=lambda (i,u): u)

[(727, 3.1810477203437779),
 (169, 3.9329042301141195),
 (610, 3.9756682216142361),
 (620, 5.1926509182451355),
 (63, 5.2855456812816088),
 (5, 5.7350827654530265),
 (165, 5.745031768581776),
 (694, 5.8661180232444305),
 (495, 5.8919548581651853),
 (274, 5.9400068981819842),
 (560, 6.048931655870593),
 (123, 6.0707415160254907),
 (155, 6.1278419175325842),
 (733, 6.2048036023908022),
 (109, 6.4700698508461052),
 (153, 6.5637289562403174),
 (745, 6.5702942267774995),
 (709, 6.5944655403305283),
 (438, 6.8088364176293759),
 (653, 6.8354572812210899),
 (38, 6.858599789110988),
 (264, 6.8955358779580118),
 (171, 6.9195181593858175),
 (85, 6.9385894476291279),
 (217, 6.9408015158091532),
 (32, 6.9477857940082082),
 (53, 6.9839387238387518),
 (77, 7.08644289591152),
 (272, 7.0955042554312442),
 (509, 7.2352146090127967),
 (596, 7.5407917842034076),
 (64, 7.6274967466057344),
 (51, 7.6405629543173106),
 (460, 7.7897686615635209),
 (562, 7.8755997991429929),
 (379, 7.9275611104814141),
 (332, 

In [114]:
unrolled[520]

array([[-0.04220254, -0.05286574, -0.08490267],
       [ 0.03624868,  0.04644958,  0.03600105],
       [-0.0202574 , -0.05936698, -0.03190433]], dtype=float32)

In [12]:
unrolled[727]/np.mean(np.abs(unrolled[727]))

array([[-1.2851795 , -1.42885292, -1.69315886],
       [ 1.37004876,  0.30140975,  0.81236625],
       [ 0.84444022, -0.99459118,  0.26995295]], dtype=float32)

In [83]:
np.multiply(1/np.mean(unrolled[73]), unrolled[73])

array([[ 1.28942895,  2.7069509 ,  5.14823055],
       [-4.16084623, -2.19202805, -8.0152998 ],
       [ 3.56173849,  3.47819924,  7.18362665]], dtype=float32)